In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import daolib.dao as dao
import daolib.dso as dso
import util.sectool as sectool
import util.operatetool as optool
import matplotlib.pyplot as plt
import pickle
import time
import os
%matplotlib inline
from tqdm import tqdm

关于['mom','emotion', 'pricevol', 'minute', 'finance', 'valuation', 'alpha191','growth']  八大类因子组合测试（共计158个因子+1个标签值）

导入Miya_Test 封装文件进行测试

In [ ]:
import Miya_Test as miya

### 数据读取

In [5]:
alpha191=readdir('D:\Data\\factor_alpha\\alpha191' )
finance=readdir('D:\Data\\factor_alpha\\finance' )
growth=readdir('D:\Data\\factor_alpha\\growth' )
minute=readdir('D:\Data\\factor_alpha\\minute' )
pricevol=readdir('D:\Data\\factor_alpha\\pricevol' )
valuation=readdir('D:\Data\\factor_alpha\\valuation' )
emotion=readdir('D:\Data\\factor_alpha\\emotion' )
mom=readdir('D:\Data\\factor_alpha\\mom' )

stock_price_df = dao.get_security_info('stock_price_info')
all_stock_list = stock_price_df.index.get_level_values(0).drop_duplicates().tolist()

#股票上市状态(只选择为0的值，不为0的值为异常值)
price_df=dao.get_security_info('stock_price_info')
trade_status_df=price_df.xs('trade_status',level=1)

start_date = dt.datetime(2007, 1, 1)
end_date = dt.datetime(2018,10,1)

In [6]:
factor_name_list = ['mom','emotion', 'pricevol', 'minute', 'finance', 'valuation', 'alpha191','growth']  

#周频数据
trade_date_w_series = pd.Series( dso.get_trade_date(start_date=start_date, end_date=end_date, period='w') )
week_list = [date for date in trade_date_w_series if dt.datetime(2009,10,20)<date and date < dt.datetime(2018,11,1) ]
week_data = read_factor_data(factors_name_list=factor_name_list, trade_date_20d_series=trade_date_w_series)
week_data['trade_status'] = trade_status_df

#### 查看数据

In [7]:
len(week_data)

159

In [17]:
 train1, test1=miya.data_input_class(week_data,week_list, 0.3, -1)

100%|████████████████████████████████████████████████████████████████████████████████| 451/451 [00:40<00:00, 11.25it/s]


In [19]:
# train1
test1

,mm_mom_10d,mm_mom_10d_weighted,mm_mom_10d_weightedexp,mm_mom_1m,mm_mom_1m_weighted,mm_mom_1m_weightedexp,mm_mom_1y_weightedexp,mm_mom_3d,mm_mom_3d_weighted,mm_mom_3d_weightedexp,...,gw_roe_chg,gw_roe_deducted_chg,gw_roe_deducted_yoy,gw_roe_yoy,gw_yoy_qfa_deductedprofit,gw_yoy_qfa_np_belongto_parcomsh,gw_yoy_qfa_roe_deducted,trade_status,rise_fall,date
000001.SZ,0.674490,0.525668,0.525668,1.111459,1.081018,0.620451,-0.286216,0.674490,0.269843,0.174030,...,0.337609,0.166564,0.800036,0.756733,0.153752,0.539291,0.331431,0.0,-1.0,2009-10-23
000002.SZ,-0.942738,-1.000268,-0.475597,-0.335883,-0.155023,-0.807230,0.197191,0.377045,-0.304214,-0.101870,...,-0.040896,-0.475362,-0.088619,-0.067952,-1.070733,-1.040288,-0.626859,0.0,-1.0,2009-10-23
000004.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-1.0,2009-10-23
000005.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-1.0,2009-10-23
000006.SZ,-0.982221,-0.271062,-0.305597,-0.300762,0.112913,-0.283342,1.357216,-0.695340,-0.318639,0.060504,...,0.990950,0.084404,-0.172142,0.536776,-0.812580,1.608968,0.467571,0.0,-1.0,2009-10-23
000007.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-1.0,2009-10-23
000008.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-1.0,2009-10-23
000009.SZ,0.132899,0.231173,-0.847932,0.211614,1.008977,-0.374406,0.773747,-0.895324,-0.816046,-1.040203,...,0.657770,0.674490,0.054270,0.002829,-1.797103,-1.487923,-1.242308,0.0,1.0,2009-10-23
000010.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-1.0,2009-10-23
000012.SZ,-0.188837,-0.088455,0.130115,0.115713,-0.248753,0.071002,-0.095223,-0.338948,-0.498089,1.040203,...,1.162040,1.016446,1.299939,1.223293,0.980816,0.608885,0.868783,0.0,-1.0,2009-10-23


### 预测

In [8]:
#分类模型
from sklearn.ensemble import RandomForestClassifier,  GradientBoostingClassifier
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

model_xgb = xgb.XGBClassifier(learning_rate=0.05, max_depth=5, )
model_lgb = lgb.LGBMClassifier(num_leaves=50,learning_rate=0.01)

C:\Users\yuanruyi\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\yuanruyi\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
import Miya_Test as miya

C:\Users\yuanruyi\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
Wfactor0_lgb3,Wfactor1_lgb3,Wtest_accuracy_lgb3,Wtrain_accuracy_lgb3=miya.predict_probo_class(week_data,model_lgb,48,week_list,0.3,-1)
Wfactor0_lgb4,Wfactor1_lgb4,Wtest_accuracy_lgb4,Wtrain_accuracy_lgb4=miya.predict_probo_class(week_data,model_lgb,48,week_list,0.4,-1)
Wfactor0_xgb4,Wfactor1_xgb4,Wtest_accuracy_xgb4,Wtrain_accuracy_xgb4=miya.predict_probo_class(week_data,model_xgb,48,week_list,0.4,-1)
Wfactor0_xgb3,Wfactor1_xgb3,Wtest_accuracy_xgb3,Wtrain_accuracy_xgb3=miya.predict_probo_class(week_data,model_xgb,48,week_list,0.3,-1)

100%|████████████████████████████████████████████████████████████████████████████████| 451/451 [01:11<00:00,  7.05it/s]
C:\Users\yuanruyi\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\yuanruyi\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\yuanruyi\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\yuanruyi\Anaconda3\lib\site-pac

### 因子保存

In [ ]:
import pickle
Wfactor1_lgb3.to_pickle("./factor_test_data/LightGBM_allfactor2/SWfactor1_lgb3")
Wfactor1_lgb4.to_pickle("./factor_test_data/LightGBM_allfactor2/SWfactor1_lgb4")
Wfactor1_xgb4.to_pickle("./factor_test_data/LightGBM_allfactor2/SWfactor1_xgb4")
Wfactor1_xgb3.to_pickle("./factor_test_data/LightGBM_allfactor2/SWfactor1_xgb3")

### 因子分析

In [ ]:
Wfactor1_lgb3_obj=factor_analyse("test1",Wfactor1_lgb3)
Wfactor1_lgb4_obj=factor_analyse("test1",Wfactor1_lgb4)

Wfactor1_xgb3_obj=factor_analyse("test1",Wfactor1_xgb3)
Wfactor1_xgb4_obj=factor_analyse("test1",Wfactor1_xgb4)

Wsamew_lgb3_pre,Wsamew_lgb3_unpre,Wunw_lgb3_pre,Wunw_lgb3_unpre=factor_stock_choose(Wfactor1_lgb3,100)
Wsamew_lgb4_pre,Wsamew_lgb4_unpre,Wunw_lgb4_pre,Wunw_lgb4_unpre=factor_stock_choose(Wfactor1_lgb4,100)

Wsamew_xgb3_pre,Wsamew_xgb3_unpre,Wunw_xgb3_pre,Wunw_xgb4_unpre=factor_stock_choose(Wfactor1_xgb3,100)
Wsamew_xgb4_pre,Wsamew_xgb4_unpre,Wunw_xgb4_pre,Wunw_xgb4_unpre=factor_stock_choose(Wfactor1_xgb4,100)

In [ ]:
show2(Wfactor1_lgb3_obj)

In [ ]:
show02(Wsamew_lgb3_pre)

### 读取文件夹数据

In [4]:
#读取文件夹数据
def readdir(dirname):  #读取文件名
    alist=[]
    for maindir, subdir, file_name_list in os.walk(dirname):
        for filename in file_name_list:
            apath = os.path.join(maindir, filename)
            file_path = os.path.split(apath) #分割出目录与文件  
            lists = file_path[1].split('.') #分割出文件与文件扩展名
            alist.extend(lists)
    return alist

def read_factor_data(factors_name_list, trade_date_20d_series):
    compound_factor_dict = {}
    alpha_factor_series = pd.Series()
    for compound_name in factors_name_list[:]:
        pro_factor_path = "D:\\Data\\factor_alpha\\" + compound_name + '\\'
        #alpha_factor_series = pd.Series()
        factor_list = globals().get(compound_name)
        for factor_name in factor_list:
#             print('读取 ', factor_name)
            data_df = pd.read_pickle(pro_factor_path + factor_name)
            begin_date = max(data_df.columns[0], trade_date_20d_series[0])
            trade_date_series = trade_date_20d_series[trade_date_20d_series >= begin_date]
            alpha_factor_series.loc[factor_name] = data_df.loc[:, trade_date_series]
    #alpha_factor_df = ftool.factor_class_generate(alpha_factor_series)
        #compound_factor_dict[compound_name] = alpha_factor_df
    return alpha_factor_series